Módulo DEAP --- uma introdução
==============================



## Introdução e importações



O módulo `DEAP` nos ajuda a construir algoritmos genéticos, fornecendo funções prontas para realizar diversas tarefas que estudamos durante o curso e muitas outras mais. Este módulo tem 3 submódulos que iremos utilizar:

-   `base`: este é o submódulo de base. É usando ele que iremos criar nossa caixa de ferramentas (`toolbox`) e nossa classe para computar a *fitness* dos indivíduos.

-   `tools`: este é o submódulo onde encontramos os operadores genéticos. Com as funções deste submódulo podemos criar indivíduos, populações, selecionar indivíduos, aplicar mutações e cruzamento.

-   `creator`: este é o submódulo que utilizaremos para criar coisas.

Para importar estes 3 submódulos basta rodar o código abaixo.



In [1]:
from deap import base
from deap import tools
from deap import creator

Vamos importar também o &ldquo;resolvedor&rdquo; de algoritmos genéticos do `DEAP`. O que este &ldquo;resolvedor&rdquo; faz ficará mais claro ao longo deste notebook.



In [2]:
from deap.algorithms import eaSimple

Agora vamos importar as funções e módulos que usaremos ao longo do notebook.



In [3]:
from pprint import pprint
import numpy as np
from funcoes_7 import gene_cb

## Problema das caixas binárias usando `DEAP` e a função `eaSimple`



Antes de iniciar o problema, vamos definir as constantes.



In [4]:
NUM_CAIXAS = 4

TAMANHO_POPULACAO = 10
NUM_GERACOES = 50
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.05
CHANCE_DE_MUTACAO_POR_GENE = 0.25
TAMANHO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

Vamos resolver o problema das caixas binárias usando o `DEAP`. O primeiro passo é definir se o problema que iremos resolver é de maximização ou de minimização. Neste caso, sabemos que o problema das caixas binárias é de maximização. Para informar isto ao `DEAP` iremos utilizar o `creator` da seguinte maneira:



In [5]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

No código acima, usamos o `creator.create` com os seguintes argumentos:

-   `"FitnessMax"`: este argumento é uma string e podemos dar o nome que quisermos. É recomendado dar o nome `"FitnessMax"` em problemas de maximização ou `"FitnessMin"` em problemas de minimização. É recomendado usar a primeira letra maiúscula.

-   `base.Fitness`: este argumento deve ser a classe de base que irá cuidar da nossa *fitness*. Não altere a menos que você saiba o que está fazendo.

-   `weights=(1.0,)`: este argumento deve ser uma tupla indicando os pesos de cada objetivo do seu problema. Se você tem apenas um objetivo e o problema é de maximização use `weights=(1.0,)`. Se você tem apenas um objetivo e o problema é de minimização use `weights=(-1.0,)`. Um erro comum é não representar este argumento como uma tupla.

Agora, vamos definir a estrutura de dados do nosso indivíduo. Para isso usamos o código abaixo.



In [6]:
creator.create("Individuo", list, fitness=creator.FitnessMax)

No código acima, usamos o `creator.create` com os seguintes argumentos:

-   `"Individuo"`: string. Pode usar a string que quiser, mas é recomendado usar uma string expressiva como `"Individuo"` ou `"Individual"`. É recomendado usar a primeira letra maiúscula.

-   `list`: este argumento é a estrutura de dados que será utilizada para representar um indivíduo. Neste caso estamos usando uma lista.

-   `fitness=creator.FitnessMax`: este argumento está relacionado com o primeiro `creator` que rodamos para descrever o *fitness*. Se tivéssemos dado outro nome para o `FitnessMax` acima, deveríamos usar o mesmo nome aqui.

O próximo passo é criar a nossa <u>caixa de ferramentas</u>. A caixa de ferramenta, como o nome sugere, vai armazenar as ferramentas que usaremos para construir indivíduos, populações e operar geneticamente sobre eles. Para criar a caixa de ferramentas fazemos o seguinte:



In [7]:
toolbox = base.Toolbox()

Afinal, o que é uma caixa de ferramentas sem ferramentas? Vamos adicionar ferramentas na nossa caixa! A estrutura para adicionar uma ferramenta é sempre a mesma:

1.  Usamos o método `register` da nossa caixa de ferramentas.
2.  O primeiro argumento do método é o nome da ferramenta. Nós que escolhemos!
3.  O segundo argumento do método é a função que será executada quando usarmos a nossa ferramenta.
4.  Os demais argumentos do `register` são os argumentos que serão passados para a função do passo 3.

Vamos ver um exemplo criando uma ferramenta que cria um indivíduo:



In [8]:
toolbox.register(
    "cria_individuo", tools.initRepeat, creator.Individuo, gene_cb, NUM_CAIXAS
)

Indo pela ordem que comentamos acima, sabemos que o nome da nossa nova ferramenta é `cria_individuo`. Quando essa ferramenta for usada, ela irá chamar a função `tools.initRepeat` com os seguintes argumentos: `creator.Individuo`, `gene_cb` e `NUM_CAIXAS`.

A função `tools.initRepeat` serve para criar um objeto através da repetição do uso de uma função. Neste exemplo, estamos criando um objeto do tipo `creator.Individuo` (que definimos lá em cima como sendo uma lista com uma certa propriedade de fitness) e para criar este objeto nós rodamos a função `gene_cb` um total de `NUM_CAIXAS` de vezes e armazenamos o resultado na lista. Veja que essa estratégia é a mesma que usamos na função `cria_candidato_cb` que criamos no `funcoes.py`.

Se mesmo com o parágrafo anterior ainda não ficou claro, veja a função `tools.initRepeat` sendo usada separadamente para ver como ela funciona.



In [9]:
uma_caixa_binaria = tools.initRepeat(list, gene_cb, NUM_CAIXAS)
print(uma_caixa_binaria)

[1, 1, 1, 1]


Vamos testar nossa ferramenta recém criada?



In [10]:
um_individuo_qualquer = toolbox.cria_individuo()
print(um_individuo_qualquer)

[0, 1, 1, 0]


Agora veremos como criar uma ferramenta para criar populações:



In [11]:
toolbox.register(
    "populacao", tools.initRepeat, list, toolbox.cria_individuo, TAMANHO_POPULACAO
)

Veja que usamos novamente o `tools.initRepeat` aqui, onde iremos armazenar em uma lista os indivíduos gerados pela ferramenta `toolbox.cria_individuo` que acabamos de criar logo acima. Vamos testar nossa nova ferramenta!



In [12]:
populacao = toolbox.populacao()
pprint(populacao)

[[0, 1, 1, 1],
 [0, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 0, 1],
 [0, 0, 1, 0],
 [1, 0, 0, 1],
 [1, 0, 0, 1],
 [0, 1, 0, 0]]


Nossa próxima tarefa é criar a ferramenta que computa o valor de *fitness*. Para isso precisamos escrever uma função que compute o valor de *fitness* de um indivíduo. Já fizemos isso! É a função `funcao_objetivo_cb` do nosso arquivo `funcoes.py`. No entanto, o `DEAP` tem uma característica muito peculiar que ele <u>requer</u> que a função objetivo retorne uma tupla, e não um número. Essa peculiaridade é do próprio `DEAP`, não temos muito o que fazer a não ser seguir o que ele manda aqui. Para reescrever a função que já tínhamos escrito para retornar uma tupla é muito fácil! Veja abaixo.



In [13]:
def funcao_objetivo_cb(individuo):
    """Computa a função objetivo no problema das caixas binárias.

    Args:
      individiuo: lista contendo os genes das caixas binárias
    """
    return (sum(individuo), )

Vamos criar a ferramenta que computa o *fitness*. Aqui daremos o nome de `evaluate` para essa ferramenta pois é o nome que o `DEAP` espera, por isso não traduzimos para o português.



In [14]:
toolbox.register("evaluate", funcao_objetivo_cb)

Faltam apenas os operadores genéticos de seleção, cruzamento e mutação! Para seleção, vamos usar a seleção por torneio! O `DEAP` já tem essa função implementada em `tools.selTournament`. Aqui daremos o nome de `select` para essa ferramenta pois é o nome que o `DEAP` espera, por isso não traduzimos.



In [15]:
toolbox.register(
    "select", tools.selTournament, tournsize=TAMANHO_TORNEIO
)

Para cruzamento vamos utilizar o cruzamento de ponto simples. Novamente, o `DEAP` já tem essa função em `tools.cxOnePoint`. Aqui daremos o nome de `mate` para essa ferramenta pois é o nome que o `DEAP` espera, por isso não traduzimos.



In [16]:
toolbox.register("mate", tools.cxOnePoint)

Finalmente, para mutação vamos usar a função de *flip bit*. Esta função é basicamente a função `mutacao_sucessiva_cb`. A chance de cada gene mutar é definida pelo parâmetro `indpb` abaixo. Aqui daremos o nome de `mutate` para essa ferramenta pois é o nome que o `DEAP` espera, por isso não traduzimos.



In [17]:
toolbox.register("mutate", tools.mutFlipBit, indpb=CHANCE_DE_MUTACAO_POR_GENE)

O `DEAP` nos permite criar facilmente um hall da fama do tamanho que quisermos. Este hall da fama guardará apenas os melhores $n$ indivíduos observados durante toda a busca (sem repetição).



In [18]:
hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

E, para encerrar, temos também como acompanhar as estatísticas da nossa busca em tempo real! Para isso usamos o `tools.Statistics`. Veja um exemplo abaixo.



In [19]:
estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("média", np.mean)
estatisticas.register("desv. padrão", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

Pronto! Temos tudo que precisamos, podemos rodar nosso algoritmo genético usando o `eaSimple`. Esta é uma função que recebe todas as informações sobre o algoritmo genético e evolui ele &ldquo;automaticamente&rdquo;.



In [20]:
populacao_inicial = toolbox.populacao()

populacao_final, log = eaSimple(
    populacao_inicial,
    toolbox,
    cxpb=CHANCE_DE_CRUZAMENTO,
    mutpb=CHANCE_DE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas,
    halloffame=hall_da_fama,
    verbose=True,
)

gen	nevals	média	desv. padrão	min	max
0  	10    	2.3  	0.640312    	1  	3  
1  	6     	2.8  	0.6         	2  	4  
2  	4     	3    	0.447214    	2  	4  
3  	3     	3.3  	0.458258    	3  	4  
4  	0     	3.7  	0.458258    	3  	4  
5  	7     	4    	0           	4  	4  
6  	3     	4    	0           	4  	4  
7  	6     	4    	0           	4  	4  
8  	7     	3.4  	1.2         	0  	4  
9  	2     	3.8  	0.4         	3  	4  
10 	4     	4    	0           	4  	4  
11 	2     	4    	0           	4  	4  
12 	6     	4    	0           	4  	4  
13 	6     	4    	0           	4  	4  
14 	8     	4    	0           	4  	4  
15 	6     	4    	0           	4  	4  
16 	6     	4    	0           	4  	4  
17 	3     	3.8  	0.6         	2  	4  
18 	6     	4    	0           	4  	4  
19 	6     	4    	0           	4  	4  
20 	6     	4    	0           	4  	4  
21 	7     	3.9  	0.3         	3  	4  
22 	6     	4    	0           	4  	4  
23 	4     	4    	0           	4  	4  
24 	8     	4    	0           	4  	4  
25 	4     	4

Podemos acessar o registro da nossa busca pela variável `log`.



In [21]:
print(log)

gen	nevals	média	desv. padrão	min	max
0  	10    	2.3  	0.640312    	1  	3  
1  	6     	2.8  	0.6         	2  	4  
2  	4     	3    	0.447214    	2  	4  
3  	3     	3.3  	0.458258    	3  	4  
4  	0     	3.7  	0.458258    	3  	4  
5  	7     	4    	0           	4  	4  
6  	3     	4    	0           	4  	4  
7  	6     	4    	0           	4  	4  
8  	7     	3.4  	1.2         	0  	4  
9  	2     	3.8  	0.4         	3  	4  
10 	4     	4    	0           	4  	4  
11 	2     	4    	0           	4  	4  
12 	6     	4    	0           	4  	4  
13 	6     	4    	0           	4  	4  
14 	8     	4    	0           	4  	4  
15 	6     	4    	0           	4  	4  
16 	6     	4    	0           	4  	4  
17 	3     	3.8  	0.6         	2  	4  
18 	6     	4    	0           	4  	4  
19 	6     	4    	0           	4  	4  
20 	6     	4    	0           	4  	4  
21 	7     	3.9  	0.3         	3  	4  
22 	6     	4    	0           	4  	4  
23 	4     	4    	0           	4  	4  
24 	8     	4    	0           	4  	4  
25 	4     	4

Podemos acessar o nosso hall da fama da seguinte maneira:



In [22]:
print(hall_da_fama.items)

[[1, 1, 1, 1]]
